In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz

--2022-02-24 09:30:34--  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272637746 (260M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop2.7.tgz’

spark-3.2.1-bin-had 100%[===================>] 260.01M   213MB/s    in 1.2s    

2022-02-24 09:30:47 (213 MB/s) - ‘spark-3.2.1-bin-hadoop2.7.tgz’ saved [272637746/272637746]



In [ ]:
!tar xf spark-3.2.1-bin-hadoop2.7.tgz


In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
from pyspark.sql.functions import udf, lit, when, date_sub
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType
from pyspark.sql.types import BooleanType, DateType, DoubleType
import json
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import Row
from datetime import datetime

# conf = SparkConf().setAppName('Colab').setMaster('local')
# sc = SparkContext(conf=spark)
sqlContext = SQLContext(spark)

/content/spark-3.2.1-bin-hadoop2.7/python/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
spark.sparkContext

<SparkContext master=local appName=Colab>

In [ ]:
# existing data set
import math

test_df = [
Row(1, "Hello!", 1643937003, '1643825926704', '2022-02-02 18:18:46 +00:00'),
Row(4, "Hello World!", 1643937003, '1643825926704', '2022-02-02 18:18:46 +00:00'),
Row(2, "Hello Spark!", 1644122255, math.nan, '2022-02-02 18:18:46 +00:00'),
Row(3, "Hello Old World!", 1643991898, '1643825926704', '2022-02-02 18:18:46 +00:00'),
Row(5, "Hello Spark!", math.nan, math.nan, '2022-02-02 18:18:46 +00:00')
]
schema_target = StructType([
StructField("id", IntegerType(), True),
StructField("attr", StringType(), True),
StructField("start_date", StringType(), True),
StructField("events_data_timestamp_unixtime_ms", StringType(), True),
StructField("dataFields_createdAt", StringType(), True)
])
test_df = sqlContext.createDataFrame(
spark.sparkContext.parallelize(test_df),
schema_target
)
test_df.show()
# df_target = df_target.withColumn("start_date", lit(df_target["start_date"]/1000))
# df_target = df_target.withColumn("start_date", df_target["start_date"].cast(StringType()))
test_df.printSchema()

+---+----------------+----------+---------------------------------+--------------------+
| id|            attr|start_date|events_data_timestamp_unixtime_ms|dataFields_createdAt|
+---+----------------+----------+---------------------------------+--------------------+
|  1|          Hello!|1643937003|                    1643825926704|2022-02-02 18:18:...|
|  4|    Hello World!|1643937003|                    1643825926704|2022-02-02 18:18:...|
|  2|    Hello Spark!|1644122255|                              NaN|2022-02-02 18:18:...|
|  3|Hello Old World!|1643991898|                    1643825926704|2022-02-02 18:18:...|
|  5|    Hello Spark!|       NaN|                              NaN|2022-02-02 18:18:...|
+---+----------------+----------+---------------------------------+--------------------+

root
 |-- id: integer (nullable = true)
 |-- attr: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- events_data_timestamp_unixtime_ms: string (nullable = true)
 |-- dataFields

In [ ]:
from pyspark.sql.functions import unix_timestamp
from pyspark.sql import functions as f

# df_target = test_df.withColumn("report_date", f.from_unixtime("start_date", "yyyy-MM-dd HH:mm:ss"))
test_df = test_df.withColumn("report_date", f.date_format(f.from_utc_timestamp(f.from_unixtime("start_date", "yyyy-MM-dd HH:mm:ss"), "America/Los_Angeles"), "dd/MM/yyyy"))
test_df = test_df.withColumn("events_data_timestamp_unixtime_ms_pst_calc", f.from_utc_timestamp(f.from_unixtime("start_date", "yyyy-MM-dd HH:mm:ss"), "America/Los_Angeles"))
test_df = test_df.withColumn("events_data_timestamp_unixtime_ms_pst_date_calc", f.from_utc_timestamp(f.from_unixtime("start_date", "yyyy-MM-dd"), "America/Los_Angeles"))
test_df = test_df.withColumn("events_data_timestamp_unixtime_ms_utc_calc", f.from_utc_timestamp(f.from_unixtime("start_date", "yyyy-MM-dd HH:mm:ss"), "UTC"))
test_df = test_df.withColumn("events_data_timestamp_unixtime_ms_utc_date_calc", f.from_utc_timestamp(f.from_unixtime("start_date", "yyyy-MM-dd"), "UTC"))
test_df.show()

+---+----------------+----------+---------------------------------+--------------------+-----------+------------------------------------------+-----------------------------------------------+------------------------------------------+-----------------------------------------------+
| id|            attr|start_date|events_data_timestamp_unixtime_ms|dataFields_createdAt|report_date|events_data_timestamp_unixtime_ms_pst_calc|events_data_timestamp_unixtime_ms_pst_date_calc|events_data_timestamp_unixtime_ms_utc_calc|events_data_timestamp_unixtime_ms_utc_date_calc|
+---+----------------+----------+---------------------------------+--------------------+-----------+------------------------------------------+-----------------------------------------------+------------------------------------------+-----------------------------------------------+
|  1|          Hello!|1643937003|                    1643825926704|2022-02-02 18:18:...| 03/02/2022|                       2022-02-03 17:10:03|        

In [ ]:
# test_df = test_df.withColumn("new_dataFields_createdAt", f.from_unixtime(f.unix_timestamp("dataFields_createdAt"),'yyyy-MM-dd HH:mm:ss').cast(StringType())) 
# test_df.show() 

In [ ]:

test_df.show()

+---+----------------+----------+---------------------------------+--------------------+-----------+------------------------------------------+-----------------------------------------------+------------------------------------------+-----------------------------------------------+
| id|            attr|start_date|events_data_timestamp_unixtime_ms|dataFields_createdAt|report_date|events_data_timestamp_unixtime_ms_pst_calc|events_data_timestamp_unixtime_ms_pst_date_calc|events_data_timestamp_unixtime_ms_utc_calc|events_data_timestamp_unixtime_ms_utc_date_calc|
+---+----------------+----------+---------------------------------+--------------------+-----------+------------------------------------------+-----------------------------------------------+------------------------------------------+-----------------------------------------------+
|  1|          Hello!|1643937003|                    1643825926704|2022-02-02 18:18:...| 03/02/2022|                       2022-02-03 17:10:03|        

In [ ]:
from pyspark.sql.functions import month , year, dayofmonth

test_df = test_df.withColumn('month',month(test_df.report_date))
test_df = test_df.withColumn('year',year(test_df.report_date))
test_df = test_df.withColumn('day',dayofmonth(test_df.report_date))
test_df[['report_date', 'month', 'year', 'day']].show()

+-------------------+-----+----+----+
|        report_date|month|year| day|
+-------------------+-----+----+----+
|2022-02-03 17:10:03|    2|2022|   3|
|2022-02-03 17:10:03|    2|2022|   3|
|2022-02-05 20:37:35|    2|2022|   5|
|2022-02-04 08:24:58|    2|2022|   4|
|               null| null|null|null|
+-------------------+-----+----+----+



In [ ]:
test_df.createOrReplaceTempView("test_df_temp")

In [ ]:
# spark.sql("select to_date(dataFields_createdAt, 'yyyy-MM-dd HH:mm:ss') as test3 from test_df_temp").show()


import pytz
from datetime import datetime
import math
# from pyspark.sql.functions import udf

def convert_time_format(val, format, tz):
    if val is None or val is math.nan:
        return None
    dt = datetime.fromtimestamp(int(val))
    dt = dt.astimezone(pytz.timezone(tz))
    return dt.strftime(format)
        
    

# convert_time_format_udf = udf(convert_time_format,StringType())  

convert_time_format('1643937003', '%Y-%m-%d %H:%M:%S', 'America/Los_Angeles')

'2022-02-03 17:10:03'

In [ ]:
test_df = test_df.withColumn(
                "dataFields_createdAt_new", convert_time_format_udf(test_df["dataFields_createdAt"], lit('%Y-%m-%d %H:%M:%S'), lit("America/Los_Angeles"))
            )
test_df.show()

+---+----------------+----------+---------------------------------+--------------------+-------------------+------------------------------------------+-----------------------------------------------+------------------------------------------+-----------------------------------------------+------------------------+
| id|            attr|start_date|events_data_timestamp_unixtime_ms|dataFields_createdAt|        report_date|events_data_timestamp_unixtime_ms_pst_calc|events_data_timestamp_unixtime_ms_pst_date_calc|events_data_timestamp_unixtime_ms_utc_calc|events_data_timestamp_unixtime_ms_utc_date_calc|dataFields_createdAt_new|
+---+----------------+----------+---------------------------------+--------------------+-------------------+------------------------------------------+-----------------------------------------------+------------------------------------------+-----------------------------------------------+------------------------+
|  1|          Hello!|1643937003|                   

In [ ]:
new_df = spark.sql("select LEFT(events_data_timestamp_unixtime_ms, 10) AS events_data_timestamp_unixtime_ms  from test_df_temp")
new_df.show()

+---------------------------------+
|events_data_timestamp_unixtime_ms|
+---------------------------------+
|                       1643825926|
|                       1643825926|
|                              NaN|
|                       1643825926|
|                              NaN|
+---------------------------------+



In [ ]:
new_df = new_df.withColumn("report_date2", f.from_utc_timestamp(f.from_unixtime("events_data_timestamp_unixtime_ms", "yyyy-MM-dd hh:mmaa"), "America/Los_Angeles"))
new_df = new_df.withColumn("events_data_timestamp_unixtime_ms_pst_calc2", f.from_utc_timestamp(f.from_unixtime("events_data_timestamp_unixtime_ms", "yyyy-MM-dd HH:mm:ss"), "America/Los_Angeles"))
new_df[['events_data_timestamp_unixtime_ms', 'report_date2', 'events_data_timestamp_unixtime_ms_pst_calc2']].show()

+---------------------------------+-------------------+-------------------------------------------+
|events_data_timestamp_unixtime_ms|       report_date2|events_data_timestamp_unixtime_ms_pst_calc2|
+---------------------------------+-------------------+-------------------------------------------+
|                       1643825926|2022-02-02 10:18:46|                        2022-02-02 10:18:46|
|                       1643825926|2022-02-02 10:18:46|                        2022-02-02 10:18:46|
|                              NaN|               null|                                       null|
|                       1643825926|2022-02-02 10:18:46|                        2022-02-02 10:18:46|
|                              NaN|               null|                                       null|
+---------------------------------+-------------------+-------------------------------------------+



In [ ]:
import re
re.split('(?=[A-Z])', "aaPaPa")

['aa', 'Pa', 'Pa']

In [ ]:
# Source data set
data_source = [
Row(1, "Hello World!"),
Row(2, "Hello PySpark!"),
Row(4, "Hello Scala!")
]
schema_source = StructType([
StructField("src_id", IntegerType(), True),
StructField("src_attr", StringType(), True)
])
df_source = sqlContext.createDataFrame(
sc.parallelize(data_source),
schema_source
)
df_source.show()
df_source.printSchema()

+------+--------------+
|src_id|      src_attr|
+------+--------------+
|     1|  Hello World!|
|     2|Hello PySpark!|
|     4|  Hello Scala!|
+------+--------------+

root
 |-- src_id: integer (nullable = true)
 |-- src_attr: string (nullable = true)



In [ ]:
high_date = datetime.strptime('9999-12-31', '%Y-%m-%d').date()
print(high_date)
current_date = datetime.today().date()
print(current_date)
# Prepare for merge - Added effective and end date
df_source_new = df_source.withColumn('src_start_date', lit(
current_date)).withColumn('src_end_date', lit(high_date))
# FULL Merge, join on key column and also high date column to make only join to the latest records
df_merge = df_target.join(df_source_new, (df_source_new.src_id == df_target.id) &
(df_source_new.src_end_date == df_target.end_date), how='fullouter')
# Derive new column to indicate the action
df_merge = df_merge.withColumn('action',
when(df_merge.attr != df_merge.src_attr, 'UPSERT')
.when(df_merge.src_id.isNull() & df_merge.is_current, 'DELETE')
.when(df_merge.id.isNull(), 'INSERT')
.otherwise('NOACTION')
)
df_merge.show()

In [ ]:
# Generate the new data frames based on action code
column_names = ['id', 'attr', 'is_current',
'is_deleted', 'start_date', 'end_date']
# For records that needs no action
df_merge_p1 = df_merge.filter(
df_merge.action == 'NOACTION').select(column_names)
# For records that needs insert only
df_merge_p2 = df_merge.filter(df_merge.action == 'INSERT').select(df_merge.src_id.alias('id'),
df_merge.src_attr.alias(
'attr'),
lit(True).alias(
'is_current'),
lit(False).alias(
'is_deleted'),
df_merge.src_start_date.alias(
'start_date'),
df_merge.src_end_date.alias(
'end_date')
)
# For records that needs to be deleted
df_merge_p3 = df_merge.filter(
df_merge.action == 'DELETE').select(column_names).withColumn('is_current', lit(False)).withColumn('is_deleted', lit(True))
# For records that needs to be expired and then inserted
df_merge_p4_1 = df_merge.filter(df_merge.action == 'UPSERT').select(df_merge.src_id.alias('id'),
df_merge.src_attr.alias(
'attr'),
lit(True).alias(
'is_current'),
lit(False).alias(
'is_deleted'),
df_merge.src_start_date.alias(
'start_date'),
df_merge.src_end_date.alias(
'end_date')
)
df_merge_p4_2 = df_merge.filter(
df_merge.action == 'UPSERT').withColumn(
'end_date', date_sub(df_merge.src_start_date, 1)).withColumn(
'is_current', lit(False)).withColumn(
'is_deleted', lit(False)).select(column_names)

In [ ]:
# Union all records together
df_merge_final = df_merge_p1.unionAll(df_merge_p2).unionAll(
df_merge_p3).unionAll(df_merge_p4_1).unionAll(df_merge_p4_2)
df_merge_final.orderBy(['id', 'start_date']).show()
# At last, you can overwrite existing data using this new data frame.
# ...

In [ ]:
!pip install boto3

     |████████████████████████████████| 131 kB 8.2 MB/s 
     |████████████████████████████████| 8.5 MB 49.4 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 138 kB 64.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install mysql-connector==2.2.9
import mysql.connector

     |████████████████████████████████| 11.9 MB 5.1 MB/s 
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp37-cp37m-linux_x86_64.whl size=247965 sha256=90df9c26b80de7db015269461e382fab1de4b4812dc862b88ff9ce9ad6d58526
  Stored in directory: /root/.cache/pip/wheels/42/2f/c3/692fc7fc1f0d8c06b9175d94f0fc30f4f92348f5df5af1b8b7
Successfully built mysql-connector


In [ ]:
! pip install snowflake-connector-python==2.7.2
from snowflake.connector import Connect

     |████████████████████████████████| 16.4 MB 27 kB/s 
     |████████████████████████████████| 2.0 MB 53.8 MB/s 
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 192 kB 52.3 MB/s 
     |████████████████████████████████| 104 kB 45.2 MB/s 
     |████████████████████████████████| 3.5 MB 30.0 MB/s 


In [ ]:
from dateutil import parser
import pytz
from pyspark.sql.functions import udf
from pyspark.sql.functions import unix_timestamp
from pyspark.sql import functions as f
from pyspark.sql.functions import month , year, dayofmonth

parser.parse("2022-02-02T06:10:17.824990Z")

datetime.datetime(2022, 2, 2, 6, 10, 17, 824990, tzinfo=tzlocal())

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/data_202202151947.csv')

In [ ]:
df.shape

(380, 19)

In [ ]:
df.query("upload_timestamp != 'skipped'", inplace = True)

In [ ]:
df.shape

(149, 19)

In [ ]:
df.sample(5)

,handle,handle_id,handle_name,video_title,video_id,upload_timestamp,video_duration,views,likes,comments,shares,profile_following,profile_follower,profile_likes,profile_total_videos,permalink_url,music_id,music_author,handle_url
103,estylecollective,NaN,E! Style Collective,"#khiryofficial came, designed, and conquered t...",7.064300e+18,34:59.5,24.262,10,1,0,1,36,234,2449,30,https://www.tiktok.com/@estylecollective/video...,6.99965E+18,I Just Wanna Know - Luke Reeves,https://www.tiktok.com/@estylecollective
83,enews,NaN,E! News,It's #nationalpizzaday and what better way to ...,7.062860e+18,32:23.4,23.798,10.4K,509,24,5,183,2M,32.8M,30,https://www.tiktok.com/@enews/video/7062859396...,7.06286E+18,original sound - E! News,https://www.tiktok.com/@enews
212,telemundo,NaN,Telemundo,Feliz juernes🎉😅 @manuelturizoz @nio garcia @an...,7.058050e+18,1/28/2022 0:00,9.448,340.2K,0,106,41,123,2M,12M,30,https://www.tiktok.com/@telemundo/video/705805...,7.05805E+18,original sound - Telemundo,https://www.tiktok.com/@telemundo
78,enews,NaN,E! News,She’ll always be Jenny from the Block. #MarryMe,7.063520e+18,31:48.3,12.049,8608,313,10,1,183,2M,32.8M,30,https://www.tiktok.com/@enews/video/7063521947...,7.04954E+18,original sound - Maggie Winters,https://www.tiktok.com/@enews
44,e_entertainment,NaN,E! Entertainment,when artificial insemination is the spark that...,7.058350e+18,1/29/2022 0:00,11.767,28.5K,1241,15,0,74,1.2M,31M,30,https://www.tiktok.com/@e_entertainment/video/...,7.05835E+18,original sound - E! Entertainment,https://www.tiktok.com/@e_entertainment
